In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("/Users/charulagnihotri/Documents/projects/fintech_breakthrough/data/reddit_cleaned.csv")

In [4]:
df.count()

score             230
timestamp         230
date              230
text              230
clean_text        230
processed_text    230
dtype: int64

In [5]:
df.head()

,score,timestamp,date,text,clean_text,processed_text
0,148,1.737167e+09,2025-01-18,"Be honest, did I miss the XRP boat? Last time ...",be honest did i miss the xrp boat last time i ...,honest miss xrp boat last time made money cryp...
1,310,1.719818e+09,2024-07-01,Convince me that Ripple (XRP) is not a Ponzi S...,convince me that ripple xrp is not a ponzi sch...,convince ripple xrp ponzi scheme big player cr...
2,1867,1.734628e+09,2024-12-19,$7 billion wiped from XRP in a day,billion wiped from xrp in a day,billion wiped xrp day
3,3486,1.689263e+09,2023-07-13,US judge just ruled XRP is not a security http...,us judge just ruled xrp is not a security \n\n...,u judge ruled xrp security u judge rule ripple...
4,668,1.740932e+09,2025-03-02,"Crypto Strategic Reserve will include XRP, SOL...",crypto strategic reserve will include xrp sol ...,crypto strategic reserve include xrp sol ada


In [6]:
# Load FinBERT tokenizer & model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:

def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)  # Get raw logits
    probs = softmax(outputs.logits, dim=1)  # Convert logits to probabilities
    
    labels = ["Negative", "Neutral", "Positive"]
    sentiment = labels[torch.argmax(probs)]  # Get the highest probability label
    return sentiment

In [8]:
df["sentiment"] = df["processed_text"].apply(get_sentiment)

In [19]:
df.to_csv("reddit_sentiment.csv", index=False)
print("✅ Sentiment analysis completed and saved to reddit_sentiment.csv")

✅ Sentiment analysis completed and saved to reddit_sentiment.csv


In [10]:
df_new = pd.read_csv("/Users/charulagnihotri/Documents/projects/fintech_breakthrough/data/reddit_sentiment.csv")

df_new.head()

,score,timestamp,date,text,clean_text,processed_text,sentiment
0,148,1.737167e+09,2025-01-18,"Be honest, did I miss the XRP boat? Last time ...",be honest did i miss the xrp boat last time i ...,honest miss xrp boat last time made money cryp...,Positive
1,310,1.719818e+09,2024-07-01,Convince me that Ripple (XRP) is not a Ponzi S...,convince me that ripple xrp is not a ponzi sch...,convince ripple xrp ponzi scheme big player cr...,Positive
2,1867,1.734628e+09,2024-12-19,$7 billion wiped from XRP in a day,billion wiped from xrp in a day,billion wiped xrp day,Positive
3,3486,1.689263e+09,2023-07-13,US judge just ruled XRP is not a security http...,us judge just ruled xrp is not a security \n\n...,u judge ruled xrp security u judge rule ripple...,Positive
4,668,1.740932e+09,2025-03-02,"Crypto Strategic Reserve will include XRP, SOL...",crypto strategic reserve will include xrp sol ...,crypto strategic reserve include xrp sol ada,Positive


In [11]:
df_new.tail()

,score,timestamp,date,text,clean_text,processed_text,sentiment
225,0,1.737932e+09,2025-01-26,Restake XRP for 6 months? I’ve just received a...,restake xrp for months ive just received a de...,restake xrp month ive received decent amount i...,Neutral
226,117,1.609488e+09,2021-01-01,Who is buying xrp right now? I don't mean this...,who is buying xrp right now i dont mean this i...,buying xrp right dont mean sarcastic dickish w...,Positive
227,60,1.680454e+09,2023-04-02,Why are XRP and XLM on a tear lately? Maybe it...,why are xrp and xlm on a tear lately maybe its...,xrp xlm tear lately maybe due banking crisis m...,Positive
228,52,1.734468e+09,2024-12-17,Bloomberg analysts expect 'a wave of cryptocur...,bloomberg analysts expect a wave of cryptocurr...,bloomberg analyst expect wave cryptocurrency e...,Positive
229,3880,1.642285e+09,2022-01-15,Ripple is funding propaganda against Bitcoin b...,ripple is funding propaganda against bitcoin b...,ripple funding propaganda bitcoin teaming gove...,Positive
